In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import webdriver_manager
import pandas as pd

options = webdriver.ChromeOptions()

#importar base de dados
dadosDF = pd.read_excel(r'buscas.xlsx')
display(dadosDF)
#abrindo google
driver = webdriver.Chrome(options=options)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,apple watch,iphone ipad,500,1000
1,rtx 4070,zotac galax,4000,4500


In [2]:
# produtos = dadosDF['Nome']
# pMin = dadosDF['Preço mínimo']
# pMax = dadosDF['Preço máximo']
# tBan = dadosDF['Termos banidos']

In [3]:
#definindo funcoes google e buscape
def verificarTermos (listatBan, nomes):
    termosBanNomes = False
    for item in listatBan:
        if item in nomes:
            termosBanNomes = True
    return termosBanNomes

def verificarTermosObrigatorios (listaProdutos, nomes):
    termosObri = True
    for item in listaProdutos:
        if item not in nomes:
            termosObri = False
    return termosObri
            
def buscaGoogle(driver, produtos, tBan, pMax, pMin):
    produtos = produtos.lower()
    tBan= tBan.lower()
    listaProdutos = produtos.split(' ')
    listatBan = tBan.split(' ')
    #criando lista de resultados vazia
    buscas = []
    #buscando produtos
    driver.get('https://shopping.google.com.br/')
    driver.find_element(By.ID, 'REsRA').send_keys(produtos, Keys.ENTER)

    #pegando informações
    resultados = driver.find_elements(By.CLASS_NAME, 'sh-dgr__content')
    for resultado in resultados:
        #tratando nome
        nomes = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nomes = nomes.lower()
        
        #se tem termos banidos
        termosBanNomes = verificarTermos(listatBan, nomes)
        #se tem todos os termos necessarios
        termosObri = verificarTermosObrigatorios(listaProdutos, nomes)
        
        #se nao tem termos banidos E tem tudo que quero
        if termosBanNomes == False and termosObri == True:
            try:
            #tratando preços 
                precos = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text 
                precos = precos.replace ('R$', '').replace('.', '').replace(',', '.')
                precos = float(precos)
            except:
                continue
            #se esta entre o preco min e o max
            if pMin <= precos <= pMax:
                refLinks = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                parLinks = refLinks.find_element(By.XPATH, '..')
                links = parLinks.get_attribute('href')
                #adicionando o resultado na lista
                buscas.append((nomes, precos, links))
    return buscas

def buscaBuscape(driver, produtos, tBan, pMax, pMin):
    produtos = produtos.lower()
    tBan= tBan.lower()
    listaProdutos = produtos.split(' ')
    listatBan = tBan.split(' ')
    #criando lista de resultados vazia
    buscas2 = []
    driver.get('https://www.buscape.com.br/')
    driver.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__HG105').send_keys(produtos, Keys.ENTER)
    #pegando informações
    resultados = driver.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard__1D3ve')
    for resultado in resultados:
        #tratando nome
        nomes = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        nomes = nomes.lower()
        links = resultado.get_attribute('href')
        
        #se tem termos banidos
        termosBanNomes = verificarTermos(listatBan, nomes)
        #se tem todos os termos necessarios
        termosObri = verificarTermosObrigatorios(listaProdutos, nomes)

        #se nao tem termos banidos E tem tudo que quero
        if termosBanNomes == False and termosObri == True:
            try:
            #tratando preços 
                precos = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text 
                precos = precos.replace ('R$', '').replace('.', '').replace(',', '.')
                precos = float(precos)
            except:
                continue

            #se esta entre o preco min e o max
            if pMin <= precos <= pMax:
                buscas2.append((nomes, precos, links))
    return buscas2

In [4]:
ofertasDF = pd.DataFrame()

for linha in dadosDF.index:
    produtos = dadosDF.loc[linha, 'Nome']
    pMin = dadosDF.loc[linha, 'Preço mínimo']
    pMax = dadosDF.loc[linha, 'Preço máximo']
    tBan = dadosDF.loc[linha, 'Termos banidos']
    
    
    ofertasGoogle = buscaGoogle(driver, produtos, tBan, pMax, pMin)
    if ofertasGoogle:
        googleDF = pd.DataFrame(ofertasGoogle, columns = ['produto', 'preço', 'link'])
        ofertasDF = pd.concat([ofertasDF, googleDF])
    else:
        googleDF = None
        
    ofertasBuscape = buscaBuscape(driver, produtos, tBan, pMax, pMin)
    if ofertasBuscape:
        buscapeDF = pd.DataFrame(ofertasBuscape, columns = ['produto', 'preço', 'link'])
        ofertasDF = pd.concat([ofertasDF, buscapeDF])
    else:
        buscapeDF = None
        
display(ofertasDF)

,produto,preço,link
0,apple watch 3 42mm | apple | usado,535.00,https://www.google.com.br/url?url=https://www....
1,apple watch series 3 (sucata) | celular apple ...,509.00,https://www.google.com.br/url?url=https://www....
2,apple watch serie 01 | apple | usado,670.00,https://www.google.com.br/url?url=https://www....
3,apple watch 3 38mm *leia a descrição* | apple ...,665.00,https://www.google.com.br/url?url=https://www....
4,apple watch s3 | apple | usado,715.00,https://www.google.com.br/url?url=https://www....
5,apple watch series 2 42mm cinza espacial,845.00,https://www.google.com.br/url?url=https://www....
6,apple watch serie 2 | apple | usado,718.00,https://www.google.com.br/url?url=https://www....
0,placa de vídeo msi geforce rtx 4070 ventus 2x ...,4399.99,https://www.google.com.br/url?url=https://www....
1,"placa de vídeo geforce rtx 4070, 12gb gddr6x, ...",4099.90,https://www.google.com.br/url?url=https://www....
2,placa de vídeo gainward rtx 4070 ghost 12gb gd...,4144.93,https://www.google.com.br/url?url=https://www....


In [6]:
#exportar excel
ofertasDF.to_excel('Ofertas.xlsx', index = False)